In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# Create images directory if it doesn't exist
os.makedirs('../images', exist_ok=True)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'


# Time ratio diff coefficients by rating band and time control
time_coef_results = {
    'rating_band': ['1000-1500', '1000-1500', '1000-1500', 
                    '1500-2000', '1500-2000', '1500-2000',
                    '2000-2500', '2000-2500', '2000-2500'],
    'time_control': ['1+0', '3+0', '5+0'] * 3,
    'time_ratio_diff_coef': [
        # 1000-1500
        0.82, 0.55, 0.35,
        # 1500-2000  
        0.74, 0.47, 0.35,
        # 2000-2500
        0.72, 0.49, 0.37
    ]
}

# All coefficients for each rating band (pick one time control, e.g., 3+0)
all_coefficients = {
    'Feature': ['elo_diff', 'material_balance', 'time_ratio_diff', 
                'white_time_ratio', 'black_time_ratio', 'move_number'],
    '1000-1500': [0.82, 0.67, 0.55, 0.17, -0.16, -0.05],
    '1500-2000': [0.92, 0.62, 0.47, 0.16, -0.17, -0.08],
    '2000-2500': [1.33, 0.68, 0.49, 0.13, -0.18, -0.08],
}

# Model performance
auc_results = {
    'Rating Band': ['1000-1500', '1500-2000', '2000-2500'],
    'AUC-ROC': [0.76, 0.77, 0.81],
    'N Samples': [20000, 20000, 18000]
}


# ============================================
# PLOT 1: Time Coefficient Comparison
# ============================================

df = pd.DataFrame(time_coef_results)
df_pivot = df.pivot(index='time_control', columns='rating_band', values='time_ratio_diff_coef')
df_pivot = df_pivot[['1000-1500', '1500-2000', '2000-2500']]
df_pivot = df_pivot.reindex(['1+0', '3+0', '5+0'])

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(df_pivot.index))
width = 0.25

colors = ["#3367BA", '#55A868', '#C44E52']

for i, (rating_band, color) in enumerate(zip(df_pivot.columns, colors)):
    bars = ax.bar(x + i * width, df_pivot[rating_band], width, 
                  label=rating_band, color=color, edgecolor='white', linewidth=0.5)
    
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Time Control', fontsize=12)
ax.set_ylabel('Time Ratio Difference Coefficient', fontsize=12)
ax.set_title('Effect of Time Advantage on Win Probability\nby Rating Band and Time Control', fontsize=14)
ax.set_xticks(x + width)
ax.set_xticklabels(df_pivot.index)
ax.legend(title='Rating Band', loc='upper right')
ax.set_ylim(0, max(df['time_ratio_diff_coef']) * 1.2)

plt.tight_layout()
plt.savefig('../images/time_coefficient_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("  Saved: ../images/time_coefficient_comparison.png")


# ============================================
# PLOT 2: Coefficient Heatmap
# ============================================


df_coef = pd.DataFrame(all_coefficients)
df_coef = df_coef.set_index('Feature')

fig, ax = plt.subplots(figsize=(8, 6))

ax.grid(False)
im = ax.imshow(df_coef.values, cmap='RdYlBu_r', aspect='auto', vmin=-1.5, vmax=1.5)

ax.set_xticks(np.arange(len(df_coef.columns)))
ax.set_yticks(np.arange(len(df_coef.index)))
ax.set_xticklabels(df_coef.columns)
ax.set_yticklabels(df_coef.index)

plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

for i in range(len(df_coef.index)):
    for j in range(len(df_coef.columns)):
        value = df_coef.iloc[i, j]
        text_color = 'white' if abs(value) > 0.8 else 'black'
        ax.text(j, i, f'{value:.2f}', ha='center', va='center', color=text_color, fontsize=11)

ax.set_title('Model Coefficients by Rating Band\n(3+0 Blitz)', fontsize=14)

cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel('Coefficient Value', rotation=-90, va='bottom')

plt.tight_layout()
plt.savefig('../images/coefficient_heatmap.png', dpi=150, bbox_inches='tight')
plt.close()
print("  Saved: ../images/coefficient_heatmap.png")


# ============================================
# PLOT 3: AUC Comparison
# ============================================

df_auc = pd.DataFrame(auc_results)

fig, ax = plt.subplots(figsize=(8, 5))

bars = ax.bar(df_auc['Rating Band'], df_auc['AUC-ROC'], color='#4C72B0', edgecolor='white')

for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=11)

ax.set_xlabel('Rating Band', fontsize=12)
ax.set_ylabel('AUC-ROC', fontsize=12)
ax.set_title('Model Performance by Rating Band for 3+0 Blitz Games', fontsize=14)
ax.set_ylim(0.4, 0.9)

ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Random (0.5)')
ax.legend()

plt.tight_layout()
plt.savefig('../images/auc_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("  Saved: ../images/auc_comparison.png")



  Saved: ../images/time_coefficient_comparison.png
  Saved: ../images/coefficient_heatmap.png
  Saved: ../images/auc_comparison.png
